## 교차 검증과 그리드 서치
+ 테스트 세트를 사용하지 않으면 모델이 과적합인지 과소적합인지 판단하기 어렵다
+ 테스트 세트를 사용하지 않고 이를 측정하는 간단한 방법은 훈련 세트를 또 나누는 것, 이를 검증 세트(validation set)라고 부른다
+ 훈련 세트에서 모델 훈련 -> 검증 세트로 모델 평가

In [1]:
import pandas as pd

In [2]:
wine = pd.read_csv('https://bit.ly/wine_csv_data')

In [3]:
wine.head()

,alcohol,sugar,pH,class
0,9.4,1.9,3.51,0.0
1,9.8,2.6,3.20,0.0
2,9.8,2.3,3.26,0.0
3,9.8,1.9,3.16,0.0
4,9.4,1.9,3.51,0.0


In [4]:
data = wine[['alcohol', 'sugar', 'pH']].to_numpy()
target = wine['class'].to_numpy()

In [5]:
from sklearn.model_selection import train_test_split

In [6]:
train_input, test_input, train_target, test_target = train_test_split(data, target, test_size = 0.2, random_state = 42)

In [7]:
sub_input, val_input, sub_target, val_target = train_test_split(train_input, train_target, train_size =  0.2, random_state = 42)

In [8]:
print(sub_input.shape, val_input.shape)

(1039, 3) (4158, 3)


In [9]:
from sklearn.tree import DecisionTreeClassifier

In [10]:
dt = DecisionTreeClassifier(random_state = 42)
dt.fit(sub_input, sub_target)
print(dt.score(sub_input, sub_target))
print(dt.score(val_input, val_target))

0.9980750721847931
0.8167388167388168


## 교차 검증
+ 검증 세트를 너무 조금 떼어 놓으면 검증 점수가 불안정
+ 이를 위해 교차 검증(cross validation) 사용
+ 검증 세트를 떼어 내어 평가하는 과정을 여러 번 반복하고, 점수를 평균하여 최종 검증 점수를 얻음

In [11]:
from sklearn.model_selection import cross_validate

In [12]:
scores = cross_validate(dt, train_input, train_target)
print(scores)

{'fit_time': array([0.00807786, 0.00406027, 0.00489712, 0.00532389, 0.00400472]), 'score_time': array([0.        , 0.        , 0.00068069, 0.        , 0.0010004 ]), 'test_score': array([0.86923077, 0.84615385, 0.87680462, 0.84889317, 0.83541867])}


+ fit_time, score_time, test_score 키를 가진 딕셔너리 반환
+ 처음 두개의 키는 훈련시간, 검증 시간 의미,
+ 교차 검증의 최종 점수는 test_score에 담긴 5개의 점수를 평균하여 얻을 수 있다

In [13]:
import numpy as np

In [14]:
print(np.mean(scores['test_score']))

0.855300214703487


In [15]:
from sklearn.model_selection import StratifiedKFold

In [16]:
splitter = StratifiedKFold(n_splits = 10, shuffle = True, random_state = 42) # n_splits는 몇겹 교차검증을 할지 지정
scores = cross_validate(dt, train_input, train_target, cv = splitter)
print(np.mean(scores['test_score']))

0.8574181117533719


## 하이퍼파라미터 튜닝
+ 사용자가 직접 지정해야 하는 파라미터를 하이퍼파라미터라고 부른다
+ 파라미터 튜닝은 기본값 그대로 사용하여 모델을 훈련하고 점증세트의 점수나 교차검증을 통해 매개변수를 조금씩 바꿔간다
+ 매개변수를 바꿔가며 모델을 훈련하고 교차 검증을 수행한다

## 그리드 서치
+ 한 매개변수가 바뀌면 다른 매개변수의 최적값도 달라지는 문제
+ 매개변수가 많아지면 문제가 더 복잡해짐
+ 이를 해결하기 위한 방법중 하나인 사이킷런의 GridSearchCV 클래스, 하이퍼파라미터 탐색과 교차검증을 한번에 수행

In [17]:
from sklearn.model_selection import GridSearchCV

In [18]:
params = {
    'min_impurity_decrease': [0.0001, 0.0002, 0.0003, 0.0004, 0.0005]
}

In [19]:
gs = GridSearchCV(DecisionTreeClassifier(random_state = 42), params, n_jobs = 1)

In [20]:
gs.fit(train_input, train_target)

,estimator,DecisionTreeC...ndom_state=42)
,param_grid,"{'min_impurity_decrease': [0.0001, 0.0002, ...]}"
,scoring,None
,n_jobs,1
,refit,True
,cv,None
,verbose,0
,pre_dispatch,'2*n_jobs'
,error_score,nan
,return_train_score,False
,criterion,'gini'


In [21]:
dt = gs.best_estimator_ # 최적의 파라미터를 선택하고, 이를 바로 사용할 수 있음
print(dt.score(train_input, train_target))

0.9615162593804117


In [22]:
print(gs.best_params_)

{'min_impurity_decrease': 0.0001}


In [23]:
print(gs.cv_results_)

{'mean_fit_time': array([0.00461869, 0.00357018, 0.00339212, 0.00219145, 0.00223904]), 'std_fit_time': array([0.00108047, 0.00061336, 0.00171508, 0.00220322, 0.00185429]), 'mean_score_time': array([0.00055232, 0.00020185, 0.00078087, 0.00056529, 0.00076075]), 'std_score_time': array([0.0004688 , 0.00040369, 0.00156174, 0.00113058, 0.00152149]), 'param_min_impurity_decrease': masked_array(data=[0.0001, 0.0002, 0.0003, 0.0004, 0.0005],
             mask=[False, False, False, False, False],
       fill_value=1e+20), 'params': [{'min_impurity_decrease': 0.0001}, {'min_impurity_decrease': 0.0002}, {'min_impurity_decrease': 0.0003}, {'min_impurity_decrease': 0.0004}, {'min_impurity_decrease': 0.0005}], 'split0_test_score': array([0.86923077, 0.87115385, 0.86923077, 0.86923077, 0.86538462]), 'split1_test_score': array([0.86826923, 0.86346154, 0.85961538, 0.86346154, 0.86923077]), 'split2_test_score': array([0.8825794 , 0.87680462, 0.87584216, 0.88161694, 0.8825794 ]), 'split3_test_score': arr

In [24]:
print(gs.cv_results_['mean_test_score'])


[0.86819297 0.86453617 0.86492226 0.86780891 0.86761605]


In [27]:
best_index = np.argmax(gs.cv_results_['mean_test_score'])
print(gs.cv_results_['params'][best_index])

{'min_impurity_decrease': 0.0001}


In [32]:
params = {
    'min_impurity_decrease' : np.arange(0.0001, 0.001, 0.0001),
    'max_depth' : range(5, 20, 1),
    'min_samples_split' : range(2, 100, 10)
}

In [33]:
gs = GridSearchCV(DecisionTreeClassifier(random_state = 42), params, n_jobs = -1)
gs.fit(train_input, train_target)

,estimator,DecisionTreeC...ndom_state=42)
,param_grid,"{'max_depth': range(5, 20), 'min_impurity_decrease': array([0.0001... 0.0009]), 'min_samples_split': range(2, 100, 10)}"
,scoring,None
,n_jobs,-1
,refit,True
,cv,None
,verbose,0
,pre_dispatch,'2*n_jobs'
,error_score,nan
,return_train_score,False
,criterion,'gini'


In [34]:
print(gs.best_params_)

{'max_depth': 14, 'min_impurity_decrease': 0.0004, 'min_samples_split': 12}


In [35]:
print(np.max(gs.cv_results_['mean_test_score']))

0.8683865773302731


## 랜덤 서치
+ 매개변수의 값이 수치일 때 값의 범위나 간격을 미리 정하기 어려울 수 있다
+ 랜덤 서치넌 매개변수 값의 목록을 전달하는 것이 아닌, 매개변수를 샘플링 할 수 있는 확률 분포 객체를 전달한다

In [38]:
from scipy.stats import uniform, randint # 주어진 범위에서 고르게 값을 뽑는다 -> 균등 분포

In [39]:
rgen = randint(0,10)
rgen.rvs(10)

array([0, 1, 0, 8, 1, 1, 6, 5, 0, 8], dtype=int64)

In [40]:
np.unique(rgen.rvs(1000), return_counts = True)

(array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=int64),
 array([105, 102,  94, 107,  92, 101, 100,  94, 110,  95], dtype=int64))

In [41]:
ugen = uniform(0,1)
ugen.rvs(10)

array([0.95439144, 0.50062484, 0.18194527, 0.50134169, 0.99792305,
       0.21229409, 0.75107586, 0.55917483, 0.71861954, 0.55948875])

In [42]:
params = {
    'min_impurity_decrease' : uniform(0.0001, 0.001),
    'max_depth' : randint(20, 50),
    'min_samples_split' : (2, 25),
    'min_samples_leaf' : (1, 25)
}

In [43]:
from sklearn.model_selection import RandomizedSearchCV

In [44]:
gs = RandomizedSearchCV(DecisionTreeClassifier(random_state = 42), params, n_iter = 100, n_jobs = -1, random_state = 42)

In [45]:
gs.fit(train_input, train_target)

,estimator,DecisionTreeC...ndom_state=42)
,param_distributions,"{'max_depth': <scipy.stats....001EE6E627D90>, 'min_impurity_decrease': <scipy.stats....001EE6E2C5350>, 'min_samples_leaf': (1, ...), 'min_samples_split': (2, ...)}"
,n_iter,100
,scoring,None
,n_jobs,-1
,refit,True
,cv,None
,verbose,0
,pre_dispatch,'2*n_jobs'
,random_state,42
,error_score,nan


In [47]:
print(gs.best_params_)

{'max_depth': 24, 'min_impurity_decrease': 0.00044507124802668297, 'min_samples_leaf': 1, 'min_samples_split': 2}


In [48]:
print(np.max(gs.cv_results_['mean_test_score']))

0.8689635004071962


In [49]:
dt = gs.best_estimator_
print(dt.score(test_input, test_target))

0.8638461538461538
